In [1]:
import time
import ROOT
import numpy as np
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from root_pandas import read_root
from matplotlib import rc
from numpy import inf

rc('text', usetex=True)

Welcome to JupyROOT 6.15/01


In [2]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

In [3]:
def getJetData(inputFiles):
    
    cols = []
    scalar = []
    scalar.append('evid')
    scalar.append('xsec')
    scalar.append('ntrials')    
    scalar.append('x')
    scalar.append('y')
    scalar.append('Q2')
    scalar.append('W2')


    #cols.append('n_total')
    #cols.append('n_charged')

    cols.append('jet_eta')
    cols.append('jet_theta')
    cols.append('jet_p')
    cols.append('jet_pt')
    cols.append('jet_qt')
    cols.append('jet_qt_up')
    cols.append('jet_qt_down')


    cols.append('jet_z')
    cols.append('jet_z_up')
    cols.append('jet_z_down')

    cols.append('jet_lab_pt')
    cols.append('jet_lab_e')
    cols.append('jet_lab_eta')
    
    lists = scalar
    dataframes = []
    for inputFilename in inputFiles:
        start = time.time()
        df = read_root(inputFilename, columns=lists+cols)#,flatten=cols)
        dataframes.append(df)
        end = time.time()
        print '\n', 'Processed', inputFilename, 'in',  '%2.1f'%(end-start), 'seconds'
    return pd.concat([x for x in dataframes])

In [4]:
def getData(inputFiles):
    
    cols = []
    scalar = []
    scalar.append('evid')
    scalar.append('xsec')
    scalar.append('ntrials')    
    scalar.append('x')
    scalar.append('y')
    scalar.append('Q2')

    
    lists = scalar
    dataframes = []
    for inputFilename in inputFiles:
        start = time.time()
        df = read_root(inputFilename, columns=lists+cols,flatten=cols)
        dataframes.append(df)
        end = time.time()
        print '\n', 'Processed', inputFilename, 'in',  '%2.1f'%(end-start), 'seconds'
    return pd.concat([x for x in dataframes])

In [5]:
def applyEventCuts(df):
    temp = df
    temp = applyCut(temp, '0.1 < y < 0.85', '0.1 < y < 0.85')
    temp = applyCut(temp, 'Q2>100', 'Q2>100')
    #temp = applyCut(temp, 'x > 0.05', 'x>0.05')
    temp.eval('logQ2= log(Q2)/2.3025850', inplace=True)
    temp.eval('logx= log(x)/2.3025850', inplace=True)
    return temp

def applyJetCuts(df):
    temp = df
    temp.eval('jet_qtnorm= jet_qt/sqrt(Q2)', inplace=True)
    temp.eval('jet_qtnorm_up= 1.01*jet_qt/sqrt(Q2)', inplace=True)
    temp.eval('jet_qtnorm_down= 0.99*jet_qt/sqrt(Q2)', inplace=True)

    df = applyCut(df,'jet_lab_pt>10.0', ' jet_pt_lab>10.0')
    df = applyCut(df,'jet_lab_eta<2.5', ' |jet_lab_eta|<2.5')

    #temp.eval('jet_qtnormjetpt= jet_qt_breit/sqrt(jet_pt)', inplace=True)
    #temp = applyCut(temp, ' jet_z>0.25', 'jet_z>0.25')
    #temp = applyCut(temp, ' jet_eta<0', 'jet eta <0')
    df = applyCut(df, 'jet_z<1.0', ' jet z<1.0')
    return temp

In [9]:
df = getData(['breit_R10.root'])
xsec = np.mean(df['xsec'])
xsec = xsec*1e9
print 'xsection %2.2f [pb]' %(xsec)
accepted = df.shape[0]
print 'accepted events' , accepted
to_cross_section = xsec/(1.0*accepted)
lumi = 100 # in units of fb-1. 
to_counts = to_cross_section*1000*lumi #1000 accounts to pass from pb to fb
##Apply selection
df = applyEventCuts(df)

##Total cross-section: 
integrated_rate    = 1000*lumi*xsec #multiply by 1000 and then 100 to get integrated yield for 100 fb-1
print 'Integrated rate %2.3f [M]' %(integrated_rate/1e6)




Processed breit_R10.root in 0.2 seconds
xsection 605.12 [pb]
accepted events 479077
0.1 < y < 0.85 406612  fraction kept: 84.9
Q2>100 390733  fraction kept: 96.1
Integrated rate 60.512 [M]


In [6]:
from matplotlib import rc
rc('text', usetex=False)
df.hist(figsize=(24,24),bins=100)
plt.show()

NameError: name 'df' is not defined

## Get Jet data apply selection

In [22]:
df_jet = {}
df_jet['R10'] = getJetData(['breit_R10.root'])
df_jet['R10'] = applyEventCuts(df_jet['R10'])
df_jet['R10'] = applyJetCuts(df_jet['R10'])

df_jet['R08'] = getJetData(['breit_R08.root'])
df_jet['R08'] = applyEventCuts(df_jet['R08'])
df_jet['R08'] = applyJetCuts(df_jet['R08'])

df_jet['R06'] = getJetData(['breit_R06.root'])
df_jet['R06'] = applyEventCuts(df_jet['R06'])
df_jet['R06'] = applyJetCuts(df_jet['R06'])


df_jet['R10_HE'] = getJetData(['breit_HE_R10.root'])
df_jet['R10_HE'] = applyEventCuts(df_jet['R10_HE'])
df_jet['R10_HE'] = applyJetCuts(df_jet['R10_HE'])

df_jet['R08_HE'] = getJetData(['breit_HE_R08.root'])
df_jet['R08_HE'] = applyEventCuts(df_jet['R08_HE'])
df_jet['R08_HE'] = applyJetCuts(df_jet['R08_HE'])

df_jet['R06_HE'] = getJetData(['breit_HE_R06.root'])
df_jet['R06_HE'] = applyEventCuts(df_jet['R06_HE'])
df_jet['R06_HE'] = applyJetCuts(df_jet['R06_HE'])


df_jet['R10_HERA'] = getJetData(['breit_HERA.root'])
df_jet['R10_HERA'] = applyEventCuts(df_jet['R10_HERA'])
df_jet['R10_HERA'] = applyJetCuts(df_jet['R10_HERA'])



Processed breit_R10.root in 1.0 seconds
0.1 < y < 0.85 406612  fraction kept: 84.9
Q2>100 390733  fraction kept: 96.1
 jet_pt_lab>10.0 12210  fraction kept: 3.1
 |jet_lab_eta|<2.5 12210  fraction kept: 100.0
 jet z<1.0 11454  fraction kept: 93.8

Processed breit_R08.root in 1.0 seconds
0.1 < y < 0.85 439886  fraction kept: 85.1
Q2>100 422566  fraction kept: 96.1
 jet_pt_lab>10.0 10489  fraction kept: 2.5
 |jet_lab_eta|<2.5 10489  fraction kept: 100.0
 jet z<1.0 9987  fraction kept: 95.2

Processed breit_R06.root in 1.4 seconds
0.1 < y < 0.85 481229  fraction kept: 85.5
Q2>100 463714  fraction kept: 96.4
 jet_pt_lab>10.0 8098  fraction kept: 1.7
 |jet_lab_eta|<2.5 8097  fraction kept: 100.0
 jet z<1.0 7802  fraction kept: 96.4

Processed breit_HE_R10.root in 1.3 seconds
0.1 < y < 0.85 399459  fraction kept: 74.8
Q2>100 383762  fraction kept: 96.1
 jet_pt_lab>10.0 12482  fraction kept: 3.3
 |jet_lab_eta|<2.5 12452  fraction kept: 99.8
 jet z<1.0 11769  fraction kept: 94.5

Processed bre

In [ ]:
import matplotlib.colors as mcolors

fig, axs = plt.subplots(ncols=1, sharey=True, figsize=(7, 4))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs
hb = ax.hist2d(df_jet.query('jet_z<1.0 and abs(jet_eta)<4.0')['jet_z'], df_jet.query('jet_z<1.0 and abs(jet_eta)<4.0')['jet_eta'], norm=mcolors.PowerNorm(0.5),bins=50)

ax.set_title('Jet z vs Jet eta (Breit Frame)')


In [ ]:
rc('text', usetex=False)
df_jet.hist(figsize=(24,24),bins=100)
plt.show()

In [ ]:
nbins = 5
minimo = 0.0
maximo = 1.0
fig = plt.figure(figsize=(8,6))
q2min = 150
query = 'jet_lab_pt>5 and Q2>%2.2f'%q2min
print ' low energy'
print df_jet['R10'].query(query)['jet_lab_pt'].mean()
print df_jet['R10'].query(query)['jet_lab_e'].mean()

print 'high energy'
print df_jet['R10_HE'].query(query)['jet_lab_pt'].mean()
print df_jet['R10_HE'].query(query)['jet_lab_e'].mean()


label = {}
label['R10'] = 'R= 1.0, 63 GeV'
label['R08'] = 'R= 0.8'
label['R06'] = 'R= 0.6'
label['R10_HE'] = 'R= 1.0, 140 GeV'
label['R08_HE'] = 'R= 0.8'
label['R06_HE'] = 'R= 0.6'

label['R10_HERA'] = 'R= 1.0, 320 GeV'

#for key in ['R10_HE','R08_HE','R06_HE']:

for key in ['R10','R10_HE','R10_HERA']:
    y, x  = np.histogram(df_jet[key].query(query)['jet_z'],bins=nbins,range=(minimo,maximo))
    xerr = (x[1:] - x[:-1])/2.0
    x = (x[1:]+x[:-1])/2
    y = y*to_cross_section
    y = y*1000*lumi
    erry = np.sqrt(y)
    integral = np.sum(y)

    plt.errorbar(x,y,  xerr = xerr, fmt='o',ls='none',ms=5)
    y_up, x  = np.histogram(df_jet[key].query(query)['jet_z_up'],bins=nbins,range=(minimo,maximo))
    xerr = (x[1:] - x[:-1])/2.0
    x = (x[1:]+x[:-1])/2
    y_up = y_up*to_cross_section
    y_up = y_up*1000*lumi

    y_do, x  = np.histogram(df_jet[key].query(query)['jet_z_down'],bins=nbins,range=(minimo,maximo))
    xerr = (x[1:] - x[:-1])/2.0
    x = (x[1:]+x[:-1])/2
    y_do = y_do*to_cross_section
    y_do = y_do*1000*lumi

    print y_up 
    print y_do
    print y_up-y_do
    print (y_up-y_do)/2.0
    error= abs(y_up-y_do)/2.0
    from matplotlib import rc
    rc('text', usetex=True)
    plt.fill_between(x, y-error, y+error,alpha=0.5,label=label[key])

plt.xlim([0,1.0])
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
lg = plt.legend(loc='best', frameon=False, fontsize=20, title=r'\mathrm{Centauro}')
title = lg.get_title()
title.set_fontsize(20)
plt.xlabel(r'$z_{\mathrm{jet}}$',fontsize=25)
plt.ylabel(r'$d\sigma/d z_{\mathrm{jet}} \times 10 \, \mathrm{fb}^{-1}$', fontsize=22)
plt.title('$10+100$ $\mathrm{GeV}$, $0.1<y<0.85$, $Q^{2}>%2.0f$ $\mathrm{ GeV^{2}}$'%q2min, fontsize=20)

plt.savefig('Centauro_zjet.png', bbox_inches='tight')
plt.savefig('Centauro_zjet.pdf', bbox_inches='tight')

 low energy
10.509996
17.48494
high energy
12.443109
21.19424
[  13136.08995362   92331.55534706  158390.93078689  403303.2232875
 1719059.46412244]
[  14399.17552608   99278.52599561  177842.44860282  515591.53067948
 1673083.14928477]
[  -1263.08557246   -6946.97064855  -19451.51781593 -112288.30739198
   45976.31483766]
[  -631.54278623  -3473.48532427  -9725.75890797 -56144.15369599
  22988.15741883]
[  85637.20181301  179484.45984702  238217.93896656  435132.97971358
 1353017.2652226 ]
[  92331.55534706  186810.35616731  262216.56484336  529359.16341933
 1304262.16212552]
[ -6694.35353406  -7325.89632029 -23998.6258768  -94226.18370576
  48755.10309708]
[ -3347.17676703  -3662.94816014 -11999.3129384  -47113.09185288
  24377.55154854]
[ 172411.18064123  199820.13756368  248070.00643178  413028.98219547
 1003521.48732202]
[178852.91706079 209040.66224266 266511.05578974 485277.47694037
 961208.12064451]
[ -6441.73641956  -9220.52467898 -18441.04935796 -72248.4947449
  42313.3666775

In [ ]:
nbins = 6
fig = plt.figure(figsize=(8,6))
query = 'jet_z>0.5 and jet_lab_pt>5 and Q2>100'
minimo = 0
maximo = 0.3
y, x  = np.histogram(df_jet.query(query)['jet_qtnorm'],bins=nbins,range=(minimo,maximo))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y*to_cross_section
y = y*1000*lumi
erry = np.sqrt(y)
plt.errorbar(x,y,  xerr = xerr, fmt='.',ls='none',label=r'jet $\eta<$0')

y_up, x  = np.histogram(df_jet.query(query)['jet_qtnorm_up'],bins=nbins,range=(minimo,maximo))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y_up = y_up*to_cross_section
y_up = y_up*1000*lumi

y_do, x  = np.histogram(df_jet.query(query)['jet_qtnorm_down'],bins=nbins,range=(minimo,maximo))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y_do = y_do*to_cross_section
y_do = y_do*1000*lumi

print y_up 
print y_do
print y_up-y_do
print (y_up-y_do)/2.0
error= abs(y_up-y_do)/2.0
from matplotlib import rc
rc('text', usetex=True)
plt.fill_between(x, y-error, y+error,alpha=0.5,label='error')
plt.fill_between(x, y*0.98, y*1.02,alpha=0.5,label='error')

plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'$q_{\mathrm{T}}/Q$',fontsize=25)
plt.ylabel(r'$d\sigma/d q_{\mathrm{T}} \times 10 \, \mathrm{fb}^{-1}$', fontsize=22)
plt.title('$\mathrm{NC}\ \mathrm{DIS}$, $10+100$ $\mathrm{GeV}$, $0.1<y<0.85$', fontsize=18)
plt.show()

In [ ]:
query = 'jet_z<0.2 and Q2>100 and jet_eta<1.0'
print df_jet.query(query)['n_total'].mean()
print df_jet.query(query)['jet_lab_pt'].mean()
print df_jet.query(query)['jet_lab_e'].mean()

# plot the z distribution of jets in current and target framengtation

In [ ]:
rc('text', usetex=True)

fig, ax1 = plt.subplots(figsize=(15,6))

##Electron distribution
plt.subplot(121)
y, x  = np.histogram(df_jet.query('jet_eta>0')['jet_z'],bins=np.linspace(0,1.0,20))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y*to_cross_section
y = y/(xerr*2.0)
plt.errorbar(x,y,  xerr = xerr, fmt='.',ls='none',label=r'jet $\eta>$0')

y, x  = np.histogram(df_jet.query('jet_eta<0')['jet_z'],bins=np.linspace(0,1.0,20))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y*to_cross_section
y = y/(xerr*2.0)
plt.errorbar(x,y,  xerr = xerr, fmt='.',ls='none',label=r'jet $\eta<$0')

plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel('jet $z$ = $2E^{jet}/Q$',fontsize=18)
plt.ylabel(r'd$\sigma$/dz [pb]',fontsize=18)
plt.title('$\mathrm{NC}\ \mathrm{DIS}$, $10+275$ $\mathrm{GeV}$, $0.1<y<0.85$', fontsize=18)

In [ ]:
## z distribution:

In [ ]:
rc('text', usetex=True)

fig, ax1 = plt.subplots(figsize=(15,6))

##Electron distribution
plt.subplot(121)
y, x  = np.histogram(df_jet.query('jet_eta>0')['jet_z'],bins=np.linspace(0,1.5,20))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y*to_cross_section
y = y/(xerr*2.0)
plt.errorbar(x,y,  xerr = xerr, fmt='.',ls='none',label=r'jet $\eta>$0')

y, x  = np.histogram(df_jet.query('jet_eta<0')['jet_z'],bins=np.linspace(0,1.5,20))
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y*to_cross_section
y = y/(xerr*2.0)
plt.errorbar(x,y,  xerr = xerr, fmt='.',ls='none',label=r'jet $\eta<$0')

plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel('jet $z$ = $2E^{jet}/Q$',fontsize=18)
plt.ylabel(r'd$\sigma$/dz [pb]',fontsize=18)
plt.title('$\mathrm{NC}\ \mathrm{DIS}$, $10+275$ $\mathrm{GeV}$, $0.1<y<0.85$', fontsize=18)


In [ ]:
from matplotlib import rc
rc('text', usetex=True)


fig, axs = plt.subplots(3, 3, sharex=True, sharey='row', figsize=(20,12),gridspec_kw={'hspace':0,'wspace':0})

for counter,edges in enumerate([(15,20),(20,25),(25,30)]):
    df_Q2cut = df_jet.query('sqrt(Q2)> %2.2f and sqrt(Q2)<%2.2f'%(edges[0],edges[1]))
    axs[0,counter].set_title(r' %2.0f$<Q<$%2.0f [GeV]'%(edges[0],edges[1]), fontsize=25)
    
    for zcounter,zedges in enumerate([(0.2,0.4), (0.4,0.6),(0.6,1.0)]):
        df_zcut = df_Q2cut.query('jet_z> %2.2f and jet_z<%2.2f'%(zedges[0],zedges[1]))
        axs[zcounter,2].yaxis.set_label_position("right")
        axs[zcounter,2].yaxis.set_label_text(' %2.1f$<2E_{jet}/Q<$%2.1f'%(zedges[0],zedges[1]),fontsize=23)
        #for xcounter,xedges in enumerate([(0.1,0.15), (0.15,0.2),(0.2,0.25),(0.25,0.30),(0.30,0.35),(0.35,0.40)]):
        for xcounter,xedges in enumerate([(0.1,0.2), (0.2,0.3),(0.3,0.4),(0.4,0.6)]):
            df_xcut = df_zcut.query('x> %2.2f and x<%2.2f'%(xedges[0],xedges[1]))
            y, x  = np.histogram(df_xcut['jet_qtnorm'],bins=6,range=(0,0.3))
            x = (x[1:]+x[:-1])/2
            y = y*to_cross_section
            y = y*1000*lumi
            erry = np.sqrt(y)
            label = '%2.1f$<x<$%2.1f'%(xedges[0],xedges[1])
            row = counter%3
            axs[zcounter,counter].errorbar(x,y,yerr=erry, fmt='-o',label=label)
            axs[zcounter,counter].semilogy()
            axs[zcounter,counter].tick_params('both',labelsize=20)

    
axs[2,0].xaxis.set_label_text(r'$q_{T}/Q$', fontsize=25)
axs[2,1].xaxis.set_label_text(r'$q_{T}/Q$', fontsize=25)
axs[2,2].xaxis.set_label_text(r'$q_{T}/Q$', fontsize=25)

axs[0,0].yaxis.set_label_text(r'$\mathrm{d \sigma}\times 100 $ fb^{-1}', fontsize=23)
axs[1,0].yaxis.set_label_text(r'$\mathrm{d \sigma}\times 100 $ fb^{-1}', fontsize=23)
axs[2,0].yaxis.set_label_text(r'$\mathrm{d \sigma}\times 100 $ fb^{-1}', fontsize=23)

axs[0,2].legend(fontsize=20,frameon=False,ncol=2)





axs[0,0].tick_params('both',labelsize=20)
#axs[0,0].xticks(fontsize=20)

    #axs[counter].text(0.1,6, , fontsize=20)
    #axs[counter].text(0.1,8, '$\sqrt{s} = 89 \, \mathrm{GeV}$ \n $0.1 < y < 0.85$ \n $p_T^{jet} > 4 \, \mathrm{GeV/c}$', fontsize=20)
    #plt.legend(prop={'size': 20}, frameon=False, loc='best')
    #axs[counter].text(0.07,10, r' $ %2.0f< p_{\mathrm{T}}^{e} < %2.0f$'%(edges[0],edges[1]) + '$\ \mathrm{ GeV}$ \n' +
                   #   r'$\langle x \rangle = %2.2f, \langle Q^{2} \rangle = %2.0f \ \mathrm{GeV}^{2}$'%(df_cut['x'].mean(), df_cut['Q2'].mean()), fontsize=25)

#axs[1].set_title(r'NC\ DIS \ $10+275$ \ $\mathrm{GeV}$, 100 $\mathrm{fb}^{-1}$ , $0.1<y<0.85$, $Q^{2}>25$ $\mathrm{GeV}^{2}$',fontsize=25)  

plt.savefig('NC_LeptonJetqt.png', bbox_inches='tight')
plt.savefig('NC_LeptonJetqt.pdf', bbox_inches='tight')